In [31]:
import pandas as pd

In [32]:
df = pd.read_csv("spotify_millsongdata.csv")

In [33]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [34]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [35]:
df.shape

(57650, 4)

In [36]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [37]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [38]:
df.head(10)

,artist,song,text
0,Who,I'm The Face,"I'm the face baby, is that clear, \r\nI'm the..."
1,Ingrid Michaelson,Open Hands,Nobody wants to be the only one left standing ...
2,Mazzy Star,Blue Light,There's a blue light in my best friend's room ...
3,Chicago,Me And My Baby,Roxie. \r\nLook at my baby my baby and me.. ...
4,Kris Kristofferson,Chase The Feeling,"It takes one to know one, baby \r\nI know how..."
5,Counting Crows,Speedway,I get so nervous I'm shaking \r\nGets so I go...
6,Jason Mraz,Be Honest,"Okay go, go hang your heart on any tree, \r\n..."
7,Electric Light Orchestra,Night In The City,Standin' at the airport \r\nLookin' down the ...
8,Dream Theater,The Test That Stumped Them All,Standing in the darkness \r\nWaiting for the ...
9,Beautiful South,Artifical Flowers,Alone in the world was poor little Anne \r\nA...


In [39]:
df['text'][0]

"I'm the face baby, is that clear,  \r\nI'm the face baby, is that clear,  \r\nI'm the face if you want it,  \r\nI'm the face if you want it, dear,  \r\nAll the others are third class tickets by me, baby, is that clear.  \r\n  \r\nI'm the big wheel baby, won't you roll with me,  \r\nI'm the big wheel baby, won't you roll with me,  \r\nSo many cats down the scene, honey,  \r\n????? hardly see.  \r\n  \r\nWear ivy league jackets, white buckskin shoes,  \r\nI wear ivy league jackets, white buckskin shoes,  \r\nSo many tickets down the scene honey,  \r\nThey're like to blow a fuse.  \r\n  \r\nI'm the face baby, is that clear,  \r\nI'm the face baby, is that clear,  \r\nI'm the face if you want it,  \r\nI'm the face if you want it, dear,  \r\nAll the others are third class tickets by me baby, is that clear.\r\n\r\n"

In [40]:
# df = df.sample(5000)

In [41]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [42]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [43]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [45]:
import nltk
nltk.download('punkt')  # Download tokenizer data


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2111y\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [47]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [50]:
similarity[0]

array([1.        , 0.0230071 , 0.        , ..., 0.        , 0.0272667 ,
       0.00424114])

In [51]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [52]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [56]:
def recommendation(song_name):
    if song_name not in df['song'].values:
        return f"Error: '{song_name}' not found in the dataset!"
    idx = df[df['song'] == song_name].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    songs = [df.iloc[m_id[0]]['song'] for m_id in distances[1:21]]
    return songs

In [57]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))